<a href="https://colab.research.google.com/github/h1-ti/GPEN/blob/main/GPEN_ENHANCER_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **実行前の準備**



1.   GOOGOLE DRIVE を開き左上の『＋新規』→　フォルダ　→　faceset フォルダを作成
2.   ローカル（自分のパソコン）の 1)Faceset/CATEGORYフォルダ を開き、超解像したいSRCをzipで圧縮する。例）1)Faceset/hogezaka/hoge -> 1)Faceset/hogezaka/hoge.zip
3.1のfacesetフォルダに2のhoge.zipをアップロード（複数可）



# **実行**



* このページの上にある　ランタイム（左から５番目）　→　すべてのセルを実行

## ※注意  **1.MOUNT GOOGLE DRIVE** のところで入力があります

### 詳しく

* GOOGLE DRIVEのマウントが完了したら終わるまで待つだけです


* 完了すると先ほどのfacesetフォルダにhoge_superres.zipができるのでドライブからダウンロードしてください


* ※srcが多すぎると最後まで完了せずに接続が切れるかもしれません。時間をおいてやり直してください。完了したものはスキップされるようになっております

* 実行中に新しいsrcのzipをアップすることも可能です


* ※また、2.INSTALL and GET MODEL WEIGHTでダウンロードがうまくいかず止まっている場合はやり直してください

**CHECK VIDEO CARD'S SPEC**


速さ比較

 P100(?f/s) > T4(6f/s) > K80(3f/s)

In [4]:
!nvidia-smi

Fri Aug 20 17:18:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **1.MOUNT GOOGLE DRIVE**

### ①Go to this URL in a browser:の右のリンクからGOOGLEにログイン
### ②Enter your authorization code:の下に①で取得したコードをペーストしてGOOGLE DRIVE をマウント



In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# **2.INSTALL and GET MODEL WEIGHT**

### install GPEN

In [6]:
%pip install ninja

%cd /content
!git clone https://github.com/h1-ti/GPEN.git

     |████████████████████████████████| 108 kB 8.1 MB/s 
/content
Cloning into 'GPEN'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (221/221), done.
remote: Total 252 (delta 41), reused 215 (delta 21), pack-reused 0
Receiving objects: 100% (252/252), 47.88 MiB | 25.88 MiB/s, done.
Resolving deltas: 100% (41/41), done.


### get weights

In [7]:
%cd /content/GPEN/weights
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-BFR-512.pth

/content/GPEN/weights
--2021-08-20 17:19:10--  https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth
Resolving public-vigen-video.oss-cn-shanghai.aliyuncs.com (public-vigen-video.oss-cn-shanghai.aliyuncs.com)... 47.101.88.25
Connecting to public-vigen-video.oss-cn-shanghai.aliyuncs.com (public-vigen-video.oss-cn-shanghai.aliyuncs.com)|47.101.88.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109497761 (104M) [application/octet-stream]
Saving to: ‘RetinaFace-R50.pth’

RetinaFace-R50.pth  100%[===================>] 104.42M  3.33MB/s    in 26s     

2021-08-20 17:19:39 (4.02 MB/s) - ‘RetinaFace-R50.pth’ saved [109497761/109497761]

--2021-08-20 17:19:39--  https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-BFR-512.pth
Resolving public-vigen-video.oss-cn-shanghai.aliyuncs.com (public-vigen-video.oss-cn-shanghai.aliyuncs.com)... 47.101.88.25
Connecting to public-vigen-video.oss-cn-shanghai.aliyuncs.com (pu

# **3.FACE ENHANCE PROCESS**


In [8]:
%cd /content/GPEN

/content/GPEN


In [ ]:
#@title FACE ENHANCE PROCESS

from face_enhancement import FaceEnhancement
from tqdm import tqdm
import os, os.path as osp
import zipfile
import glob
import cv2
import shutil
import numpy as np

from DFLIMG.DFLPNG import DFLPNG
from DFLIMG.DFLJPG import DFLJPG

root = "/content/GPEN"

model = {'name':'GPEN-BFR-512', 'size':512}
    
faceset_dir = "/content/drive/MyDrive/faceset"
indir = osp.join(root, "temp_input")
os.makedirs(indir, exist_ok=True)

faceenhancer = FaceEnhancement(base_dir=root, size=model['size'], model=model['name'], channel_multiplier=2)

done = []

while True:
    src_names = [osp.splitext(d)[0] for d in os.listdir(faceset_dir)
             if osp.splitext(d)[1] == ".zip" and "_superres" not in d
             and not osp.exists(osp.join(faceset_dir, osp.splitext(d)[0]+"_superres.zip"))
             and osp.splitext(d)[0] not in done]
    if len(src_names) == 0:
        break
    print(src_names)
    


    for id, src in enumerate(src_names):
        print("{} / {} [{}]".format(id+1, len(src_names), src))
        zip_path = osp.join(faceset_dir, src+".zip")
        print("Extracting [{}] ...".format(zip_path))
        with zipfile.ZipFile(zip_path) as z:
            z.extractall(indir)

        files = sorted(glob.glob(osp.join(indir, src, '*.*g')))

        outdir = osp.join(root, src+"_superres", src+"_superres")
        os.makedirs(outdir, exist_ok=True)

        for n, file in enumerate(tqdm(files[:], total=len(files), desc="Enhancing faces ...")):
            filename = osp.basename(file)
            ext = osp.splitext(file)[1]
            if ext == ".jpg":
                dflimg = DFLJPG.load(file)
            elif ext == ".png":
                dflimg = DFLPNG.load(file)
            else:
                continue

            if not dflimg:
                continue
            
            im = cv2.imread(file, cv2.IMREAD_COLOR) # BGR
            input_x, input_y = im.shape[:2]
            if not isinstance(im, np.ndarray): print(filename, 'error'); continue
            im = cv2.resize(im, (0,0), fx=2, fy=2)

            img, orig_faces, enhanced_faces = faceenhancer.process(im)
            img = cv2.resize(img, (input_x, input_y))
            cv2.imwrite(osp.join(outdir, filename), img)

            if ext == ".jpg":
                _fanseg_mask = dflimg.dfl_dict.get('fanseg_mask', None)
                if _fanseg_mask is not None:
                    ret, buf = cv2.imencode( '.jpg', _fanseg_mask, [int(cv2.IMWRITE_JPEG_QUALITY), 85] )

                    if ret and len(buf) < 64000:
                        _fanseg_mask = buf
                    else:
                        io.log_err("Unable to encode fanseg_mask for %s" % (filename) )
                        _fanseg_mask = None

                _xseg_mask = dflimg.dfl_dict.get('xseg_mask', None)
                if _xseg_mask is not None:
                    ret, buf = cv2.imencode( '.jpg', _xseg_mask, [int(cv2.IMWRITE_JPEG_QUALITY), 85] )

                    if ret and len(buf) < 64000:
                        _xseg_mask = buf
                    else:
                        io.log_err("Unable to encode xseg_mask for %s" % (filename) )
                        _xseg_mask = None

                DFLJPG.embed_dfldict (osp.join(outdir, filename), 
                                        {'face_type': dflimg.get_face_type(),
                                            'landmarks': dflimg.get_landmarks(),
                                            'ie_polys' : dflimg.get_ie_polys(),
                                            'source_filename': dflimg.get_source_filename(),
                                            'source_rect': dflimg.get_source_rect(),
                                            'source_landmarks': dflimg.get_source_landmarks(),
                                            'image_to_face_mat': dflimg.get_image_to_face_mat(),
                                            'fanseg_mask' : _fanseg_mask,
                                            'xseg_mask' : _xseg_mask,
                                            'eyebrows_expand_mod' : None,
                                            'relighted' : None,
                                            "histgram" : None,
                                            "recognition" : dflimg.get_recognition(),
                                        })
            elif ext == ".png":
                dflimg = DFLPNG.embed_dfldict (osp.join(outdir, filename), 
                                        {'face_type': dflimg.get_face_type(),
                                            'landmarks': dflimg.get_landmarks(),
                                            'ie_polys' : dflimg.get_ie_polys(),
                                            'source_filename': dflimg.get_source_filename(),
                                            'source_rect': dflimg.get_source_rect(),
                                            'source_landmarks': dflimg.get_source_landmarks(),
                                            'image_to_face_mat': dflimg.get_image_to_face_mat(),
                                            'fanseg_mask' : dflimg.dfl_dict.get ('fanseg_mask', None),
                                            'xseg_mask' : dflimg.dfl_dict.get('xseg_mask', None),
                                            'eyebrows_expand_mod' : None,
                                            'relighted' : None,
                                            "histgram" : None,
                                            "recognition" : dflimg.get_recognition(),
                                        })


        
        shutil.make_archive(src+"_superres", 'zip', src+"_superres")
        if osp.exists(osp.join(faceset_dir, src+"_superres.zip")):
            os.remove(osp.join(faceset_dir, src+"_superres.zip"))
        shutil.move(src+"_superres.zip", faceset_dir)
        shutil.rmtree(src+"_superres")
        print("Successfully completed {}".format(src+"_superres"))
        done.append(src)

print(done)

['AshidaMana']
1 / 1 [AshidaMana]
Extracting [/content/drive/MyDrive/faceset/AshidaMana.zip] ...


Enhancing faces ...:   0%|          | 0/7619 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Enhancing faces ...:  39%|███▉      | 3003/7619 [08:07<12:24,  6.20it/s]